# Student Loan Risk with Deep Learning

Note: Google Colab's AI was used to write some of this code, though it was edited and implemented by a human.

Also: I will be trying to run this code using the "T4 GPU" runtime in google
colab. Feel free to adjust the code to use regular runtime. Just delete the
"with tf.device('/device:GPU:0'):" parts of the primary code or edit to
a gpu that works on the system used to run this.

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

In [ ]:
#attenpting to see if I can use the gpu for this
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#Comparing speed between cpu and gpu
import timeit

def cpu():
    with tf.device('/cpu:0'):
        random_image_cpu = tf.random.normal((100, 100, 100, 3))
        net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
        return tf.math.reduce_sum(net_cpu)

def gpu():
    with tf.device('/device:GPU:0'):
        random_image_gpu = tf.random.normal((100, 100, 100, 3))
        net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
        return tf.math.reduce_sum(net_gpu)

# Warm up by running each op once
cpu()
gpu()

# Run the op several times
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time / gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.9161413350002476
GPU (s):
0.11223522499994942
GPU speedup over CPU: 34x


In [ ]:
#gpu looks to be a lot faster, so I will try to use that one for this assignment

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()
print(X_train.head())
# Fit the scaler to the features training dataset
scaler.fit(X_train)

#I assume we also scale the data:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled)

      payment_history  location_parameter  stem_degree_score  gpa_ranking  \
1144              7.3               0.320               0.23          2.3   
73                8.3               0.675               0.26          2.1   
446              12.5               0.380               0.60          2.6   
399               8.7               0.765               0.22          2.3   
647               8.3               0.845               0.01          2.2   

      alumni_success  study_major_code  time_to_completion  \
1144           0.066              35.0                70.0   
73             0.084              11.0                43.0   
446            0.081              31.0                72.0   
399            0.064               9.0                42.0   
647            0.070               5.0                14.0   

      finance_workshop_score  cohort_ranking  total_loan_score  \
1144                 0.99588            3.43              0.62   
73                   0.99760    

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)


# Review the number of features
print(input_nodes)

11


In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer
output_nodes = 1

In [ ]:
with tf.device('/device:GPU:0'):

  # Create the model
  model = tf.keras.models.Sequential()

  # First hidden layer
  model.add(Dense(units=hidden_nodes_layer1, input_dim=input_nodes, activation="relu"))

  # Second hidden layer
  model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

  # Output layer
  model.add(Dense(units=1, activation="sigmoid"))


In [ ]:
# Display the Sequential model summary
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 64)                768       
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
with tf.device('/device:GPU:0'):
  # Compile the model
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
  # Fit the model
with tf.device('/device:GPU:0'):
  model.fit(
      x=X_train_scaled,
      y=y_train,
      epochs=50,
      validation_data=(X_test, y_test),
      verbose=1,
  )

Epoch 1/50
38/38 [==============================] - 2s 8ms/step - loss: 0.0931 - accuracy: 0.9708 - val_loss: 48.3241 - val_accuracy: 0.4725
Epoch 2/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0925 - accuracy: 0.9708 - val_loss: 35.0698 - val_accuracy: 0.4825
Epoch 3/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0963 - accuracy: 0.9733 - val_loss: 49.8752 - val_accuracy: 0.4725
Epoch 4/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0891 - accuracy: 0.9775 - val_loss: 49.3179 - val_accuracy: 0.4750
Epoch 5/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0893 - accuracy: 0.9775 - val_loss: 53.7224 - val_accuracy: 0.4725
Epoch 6/50
38/38 [==============================] - 0s 4ms/step - loss: 0.0964 - accuracy: 0.9741 - val_loss: 52.3248 - val_accuracy: 0.4725
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.0872 - accuracy: 0.9758 - val_loss: 51.9479 - val_accuracy: 0.4725
Epoch 8/50
38

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
with tf.device('/device:GPU:0'):
  model_loss, model_accuracy = model.evaluate(
      x=X_test_scaled, y=y_test, verbose=2)

13/13 - 0s - loss: 1.2389 - accuracy: 0.7625 - 135ms/epoch - 10ms/step


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path

#note: I like using the default directory available in google drive, so I will
#just leave this (somewhat useless) code here. Feel free to edit it to preference
import os
current_working_directory = os.getcwd()
filepath = os.path.join(current_working_directory, 'student_loans.keras')

# Export your model to a keras file
model.save("student_loans.keras")

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
filepath = os.path.join(current_working_directory, 'student_loans.keras')

# Load the model to a new object
model = tf.keras.models.load_model("student_loans.keras")

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
with tf.device('/device:GPU:0'):
  test_predictions = model.predict(X_test_scaled)


# Display a sample of the predictions
test_predictions[0:5]

13/13 [==============================] - 0s 2ms/step


array([[9.0358069e-04],
       [1.9922120e-02],
       [9.7365618e-01],
       [6.6531450e-01],
       [9.9999964e-01]], dtype=float32)

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(test_predictions)
predictions_df = predictions_df.round()
predictions_df.head(20)

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,0.0
8,1.0
9,0.0


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75       188
           1       0.78      0.76      0.77       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loan options, gathering specific data is crucial. This includes general credit score data to assess creditworthiness, the student's chosen major to gauge potential future income, household income or a proxy for financial stability, an indicator of academic aptitude or diligence, loan term length preferences, age, and marital status. For graduate students, their income should also be considered. These data points collectively provide a comprehensive understanding of the student's financial situation, academic performance, and future earning potential, enabling the recommendation system to suggest loan options tailored to their individual needs and circumstances.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Based on the data outlined in the recommendation system, the most suitable filtering method would indeed be content-based filtering. This choice aligns with the traditional approach of analyzing data based on specified features or variables, which include the student's major, household income, academic aptitude, loan term length, age, marital status, and potentially graduate student income. Content-based filtering allows for leveraging these specific data points to make tailored recommendations without relying on comparisons or similarities between users, as in collaborative filtering. Additionally, context-based filtering, which is typically used for quick analyses, doesn't suit the nature of loan processes, which usually involve longer timelines and more comprehensive data evaluation. While collaborative filtering could find some application by identifying similarities between users, it deviates from the proposed method of utilizing specific data points. However, employing a content-based filter does raise the concern of individuals learning to manipulate the system once its inner workings are revealed, highlighting a potential drawback to be addressed in implementation.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Two significant challenges in building a recommendation system for student loans revolve around fairness and model efficacy. Ensuring fairness is paramount to prevent biases in recommendations based on variables like credit score, income, and academic performance. Without proper control or supplementation, these factors could unintentionally disadvantage certain groups. Additionally, innovating beyond conventional modeling approaches presents a challenge, as simply incorporating additional data may not significantly enhance performance. Exploring lifestyle information, though potentially beneficial, poses ethical and logistical challenges. While leveraging AI offers promise, determining its superiority over simpler machine learning methods requires careful consideration and experimentation to ensure the system's effectiveness and fairness in recommending student loan options.